In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [7]:
# Function to parse the TFRecord file
def parse_tfrecord(example_proto):
    feature_description = {
        "coordinates": tf.io.FixedLenFeature([2 * 21 * 3], tf.float32),  # Flattened 2x21x3 tensor
        "label": tf.io.FixedLenFeature([], tf.int64)
    }
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    coordinates = tf.reshape(parsed_features["coordinates"], (2, 21, 3))  # Reshape back to 2x21x3
    # label = parsed_features["label"]
    label = tf.one_hot(parsed_features["label"], depth=21)
    return coordinates, label

# Load the dataset from the TFRecord file
loaded_dataset = tf.data.TFRecordDataset("E:\Folder_mata_kuliah\Semester_7\PRA TA\Python prep\workspace_jauh\dataset_without_mirror_without_z_also_21class.tfrecords")
loaded_dataset = loaded_dataset.map(parse_tfrecord)



In [8]:
dataset_list = list(loaded_dataset.take(20))

# Examine the first example
for x in range(20):
    first_coordinates, first_label = dataset_list[x]
    print("First coordinates shape:", first_coordinates.shape)
    print("First label:", first_label.numpy())
    print("Sample coordinate values:")
    print(first_coordinates[:, 0:21, :].numpy())  # First instance, first 3 keypoints

First coordinates shape: (2, 21, 3)
First label: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Sample coordinate values:
[[[0.28163397 0.7581269  0.        ]
  [0.23312777 0.7472053  0.        ]
  [0.18026629 0.7517967  0.        ]
  [0.14359578 0.75918996 0.        ]
  [0.10979708 0.7651197  0.        ]
  [0.18408158 0.78254133 0.        ]
  [0.16124927 0.8055031  0.        ]
  [0.14523056 0.81935596 0.        ]
  [0.13066188 0.82963777 0.        ]
  [0.21442948 0.7925636  0.        ]
  [0.19265254 0.8194172  0.        ]
  [0.1821034  0.82558864 0.        ]
  [0.17269349 0.8267382  0.        ]
  [0.2491273  0.79672617 0.        ]
  [0.2333582  0.82099646 0.        ]
  [0.22949037 0.82124573 0.        ]
  [0.2238509  0.81748575 0.        ]
  [0.28294545 0.7965238  0.        ]
  [0.2784026  0.81202626 0.        ]
  [0.27671748 0.80974483 0.        ]
  [0.27534688 0.8042399  0.        ]]

 [[0.6315812  0.8404442  0.        ]
  [0.60777456 0.8153839  0.        ]
  [0.57

In [9]:
coordinates = []
labels = []
for x, y in loaded_dataset:
    coordinates.append(x.numpy())  # Append coordinates as NumPy arrays
    labels.append(y.numpy())  # Append labels as integers

In [10]:
split = 0.8
train_coordinates = np.array(coordinates[:int(len(coordinates)*split)])
train_labels = np.array(labels[:int(len(labels)*split)])

valid_coordinates = np.array(coordinates[int(len(coordinates)*split):])
valid_labels = np.array(labels[int(len(labels)*split):])

In [11]:
print(train_coordinates.shape)
print(train_labels.shape)
print(valid_coordinates.shape)

(19976, 2, 21, 3)
(19976, 21)
(4995, 2, 21, 3)


In [18]:
model = tf.keras.models.Sequential([
    # Input: 2 hands, 21 landmarks per hand, 3 coordinates per landmark
    tf.keras.layers.Input(shape=(2, 21, 3)),
    
    # Reshape for convolution (treating hands as channels)
    tf.keras.layers.Reshape((21, 3, 2)),
    
    # Convolutional layers to capture spatial relationships
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 1)),
    
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 1)),
    
    # Flatten after convolutions
    tf.keras.layers.Flatten(),
    
    # Dense layers with residual connections
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    # Output layer
    tf.keras.layers.Dense(20, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 21, 3, 2)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 3, 64)         1216      
                                                                 
 batch_normalization_5 (Batc  (None, 21, 3, 64)        256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 3, 64)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 3, 128)        73856     
                                                                 
 batch_normalization_6 (Batc  (None, 10, 3, 128)      

In [14]:
from tensorflow.keras.applications import MobileNetV3Large

# Your preprocessing pipeline
inputs = tf.keras.Input(shape=(2, 21, 3))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(32*32, activation='relu')(x)
x = tf.keras.layers.Reshape((32, 32, 1))(x)
x = tf.repeat(x, 3, axis=-1)
x = tf.keras.applications.mobilenet_v3.preprocess_input(x)
base_model =MobileNetV3Large(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Freeze all layers in base_model
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(21, activation="softmax")(x) 

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.006),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 2, 21, 3)]        0         
                                                                 
 flatten_2 (Flatten)         (None, 126)               0         
                                                                 
 dense_4 (Dense)             (None, 1024)              130048    
                                                                 
 reshape_2 (Reshape)         (None, 32, 32, 1)         0         
                                                                 
 tf.repeat_2 (TFOpLambda)    (None, 32, 32, 3)         0         
                                                                 
 MobilenetV3large (Functiona  (None, 1, 1, 960)        2996352   
 l)                                                              
                                                           

In [9]:
# del model

In [16]:
# Count total samples
total_count = sum(1 for _ in loaded_dataset)
train_size = int(0.8 * total_count)

# Shuffle before splitting
shuffled_dataset = loaded_dataset.shuffle(buffer_size=total_count, seed=9698)

# Use enumerate and filter to split
train_ds = shuffled_dataset.enumerate().filter(lambda i, _: i < train_size).map(lambda i, data: data).batch(32)
valid_ds = shuffled_dataset.enumerate().filter(lambda i, _: i >= train_size).map(lambda i, data: data).batch(32)

history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=50
) 

Epoch 1/50
625/625 [==============================] - 11s 17ms/step - loss: 0.9126 - accuracy: 0.6697 - val_loss: 0.7719 - val_accuracy: 0.7347
Epoch 2/50
625/625 [==============================] - 11s 18ms/step - loss: 0.6879 - accuracy: 0.7502 - val_loss: 0.5880 - val_accuracy: 0.7724
Epoch 3/50
625/625 [==============================] - 11s 18ms/step - loss: 0.5658 - accuracy: 0.7967 - val_loss: 0.5670 - val_accuracy: 0.7964
Epoch 4/50
625/625 [==============================] - 11s 18ms/step - loss: 0.4774 - accuracy: 0.8311 - val_loss: 0.4271 - val_accuracy: 0.8410
Epoch 5/50
625/625 [==============================] - 11s 17ms/step - loss: 0.4356 - accuracy: 0.8479 - val_loss: 0.4248 - val_accuracy: 0.8529
Epoch 6/50
625/625 [==============================] - 11s 17ms/step - loss: 0.3952 - accuracy: 0.8673 - val_loss: 0.3440 - val_accuracy: 0.8789
Epoch 7/50
625/625 [==============================] - 11s 18ms/step - loss: 0.3736 - accuracy: 0.8731 - val_loss: 0.4623 - val_accuracy:

In [17]:
model.save("MobileNetV3Large_50epoch_21_class.h5")
